<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados interativa. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/) e seu *dashboard* [Google Data Studio](https://datastudio.google.com/).

---

# **COVID Dashboard**

## 1\. Contexto

> A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global. Fonte: Governo brasileiro ([link](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus)).

Com isto, irei avaliar a tendência e o número de casos. Sendo assim também irei detalhar através do dashboard no mapa as tendências para conseguir realizar um comparativo com outros estados.

## 1.1. Dashboard (LookerStudio)
Segue o link: https://lookerstudio.google.com/s/tQrhTqaZcQc


## 2\. Pacotes e bibliotecas

In [ ]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## 3\. Extração

In [ ]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv'
dados = pd.read_csv(url, sep=',')

dados.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


In [ ]:
def date_range(start_date: datetime, end_date: datetime):
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    return start_date + timedelta(lag)

In [ ]:
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

In [ ]:
dados = None
dados_vazios = True

date_range_list = [date for date in pd.date_range(start=start_date, end=end_date)]

for date in date_range_list:

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(columns = ['FIPS', 'Admin2', 'Last_Update', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'])
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if dados_vazios:
    dados = case
    dados_vazios = False
  else:
    dados = pd.concat([dados, case], ignore_index=True)

In [ ]:
dados.query('Province_State == "Sao Paulo"').head()

,Province_State,Country_Region,Lat,Long_,Confirmed,Deaths,Incident_Rate,Date
24,Sao Paulo,Brazil,-23.5505,-46.6333,1466191,46775,3192.990778,2021-01-01
51,Sao Paulo,Brazil,-23.5505,-46.6333,1467953,46808,3196.827966,2021-01-02
78,Sao Paulo,Brazil,-23.5505,-46.6333,1471422,46845,3204.382565,2021-01-03
105,Sao Paulo,Brazil,-23.5505,-46.6333,1473670,46888,3209.278136,2021-01-04
132,Sao Paulo,Brazil,-23.5505,-46.6333,1486551,47222,3237.329676,2021-01-05


## 4\. Transformação

In [ ]:
dados.head()

,Province_State,Country_Region,Lat,Long_,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,-9.0238,-70.8120,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,-9.5713,-36.7820,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,0.9020,-52.0030,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,-3.4168,-65.8561,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,-12.5797,-41.7007,494684,9159,3326.039611,2021-01-01


In [ ]:
dados.shape

(9855, 8)

In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9855 entries, 0 to 9854
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9855 non-null   object        
 1   Country_Region  9855 non-null   object        
 2   Lat             9855 non-null   float64       
 3   Long_           9855 non-null   float64       
 4   Confirmed       9855 non-null   int64         
 5   Deaths          9855 non-null   int64         
 6   Incident_Rate   9855 non-null   float64       
 7   Date            9855 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 616.1+ KB


In [ ]:
#Renomeando colunas
dados = dados.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in dados.columns:
  dados = dados.rename(columns={col: col.lower()})

In [ ]:
#Ajustando o nome dos estados

states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo',
    'Distrito Federal': 'Brasília'
}

dados['state'] = dados['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)



*   Chaves temporais



In [ ]:
dados['month'] = dados['date'].apply(lambda date: date.strftime('%Y-%m'))
dados['year']  = dados['date'].apply(lambda date: date.strftime('%Y'))



*   População estimada do estado


In [ ]:
dados['population'] = round(100000 * (dados['confirmed'] / dados['incident_rate']))
dados = dados.drop('incident_rate', axis=1)

*   Número, média móvel (7 dias) e
estabilidade (14 dias) de casos e mortes por estado:

In [ ]:
dados_ = None
dados_vazios = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:
    status = 'diminuição'
  elif rate > 1.15:
    status = 'aumento'
  else:
    status = 'estável'

  return status


for state in dados['state'].drop_duplicates():

  dados_por_estado = dados.query(f'state == "{state}"').reset_index(drop=True)
  dados_por_estado = dados_por_estado.sort_values(by=['date'])

  dados_por_estado['confirmed_1d'] = dados_por_estado['confirmed'].diff(periods=1)
  dados_por_estado['confirmed_moving_avg_7d'] = np.ceil(dados_por_estado['confirmed_1d'].rolling(window=7).mean())
  dados_por_estado['confirmed_moving_avg_7d_rate_14d'] = dados_por_estado['confirmed_moving_avg_7d']/dados_por_estado['confirmed_moving_avg_7d'].shift(periods=14)
  dados_por_estado['confirmed_trend'] = dados_por_estado['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  dados_por_estado['deaths_1d'] = dados_por_estado['deaths'].diff(periods=1)
  dados_por_estado['deaths_moving_avg_7d'] = np.ceil(dados_por_estado['deaths_1d'].rolling(window=7).mean())
  dados_por_estado['deaths_moving_avg_7d_rate_14d'] = dados_por_estado['deaths_moving_avg_7d']/dados_por_estado['deaths_moving_avg_7d'].shift(periods=14)
  dados_por_estado['deaths_trend'] = dados_por_estado['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if dados_vazios:
    dados_ = dados_por_estado
    dados_vazios = False
  else:
    dados_ = pd.concat([dados_, dados_por_estado], ignore_index=True)

dados = dados_
dados_ = None

In [ ]:
#Fazendo type casting para garantir o tipo de dado

dados['population'] = dados['population'].astype('Int64')
dados['confirmed_1d'] = dados['confirmed_1d'].astype('Int64')
dados['confirmed_moving_avg_7d'] = dados['confirmed_moving_avg_7d'].astype('Int64')
dados['deaths_1d'] = dados['deaths_1d'].astype('Int64')
dados['deaths_moving_avg_7d'] = dados['deaths_moving_avg_7d'].astype('Int64')

In [ ]:
#Reorganizando as colunas

dados = dados[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [ ]:
dados.head(n=25)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,2021-01,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,2021-01,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,2021-01,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,2021-01,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,2021-01,2021
5,2021-01-06,Brazil,Acre,881935,42378,208,<NA>,NaN,NaN,814,6,<NA>,NaN,NaN,2021-01,2021
6,2021-01-07,Brazil,Acre,881935,42478,100,<NA>,NaN,NaN,821,7,<NA>,NaN,NaN,2021-01,2021
7,2021-01-08,Brazil,Acre,881935,42814,336,161,NaN,NaN,823,2,4,NaN,NaN,2021-01,2021
8,2021-01-09,Brazil,Acre,881935,42908,94,139,NaN,NaN,823,0,4,NaN,NaN,2021-01,2021
9,2021-01-10,Brazil,Acre,881935,43127,219,155,NaN,NaN,825,2,4,NaN,NaN,2021-01,2021


## 5\. Carregamento

In [ ]:
# faça o código para persistência dos dados:
#
# - salve-os no formato csv sem índice;
# - etc.

dados.to_csv('./covid-dados.csv', sep=',', index=False)